# Project setup

In [1]:
dataset_zip_name = "mikroszkop-bioszenzor"
dataset_name = "20210526_LCLC_fn"
model_name = "cyto3_old_annotated"

In [ ]:
!git clone --recursive -j8 https://github.com/Nanobiosensorics/cell-aligner.git
!mv cell-aligner/* ./
!rm -rf cell-aligner

In [ ]:
!pip install -r requirements.txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "/content/drive/MyDrive/szakdolgozat/kepillesztes/{dataset_zip_name}.zip" -d "/content/datasets"
!mkdir models
!cp "/content/drive/MyDrive/szakmai gyakorlat/cellpose-cell-counting/models/{model_name}" "/content/models"

In [6]:
import os

file_path = "./nanobio_core/epic_cardio/data_correction.py"
backup_file_path = "./nanobio_core/epic_cardio/data_correction.bak"

if not os.path.exists(backup_file_path):
  os.rename(file_path, backup_file_path)

  with open(backup_file_path, 'r') as file:
    lines = file.readlines()

  with open(file_path, 'w') as file:
    for line in lines:
      if "matplotlib.use('Qt5Agg')" in line:
        file.write("# " + line)
      else:
        file.write(line)

# Preprocessing

In [ ]:
import pickle
from pipeline import run_pipeline
from preprocessing import NanoReader

In [ ]:
# no magnification
file_path = "/content/drive/MyDrive/szakdolgozat/kepillesztes/preprocessed_1x.pkl"

with open(file_path, "wb") as file:
  result = run_pipeline(
      data=NanoReader(
          base_path=f"/content/datasets/{dataset_name}",
          flip_epic=[False, True]
      ),
      cellpose_model_path=f"/content/models/{model_name}",
      epic_params={
        "preprocessing_params": {
            "magnification": 1,
            "scaling": "MIC_10X",
            "range_lowerbound": 0,
            "drift_correction": {
                "threshold": 75,
                "filter_method": "mean"
            }
        },
        "localization_params": {
            "threshold_range": [150, 3000],
            "neighbourhood_size": 3
        }
      },
      only_process=True
  )
  pickle.dump(result, file)

In [ ]:
# magnification
file_path = "/content/drive/MyDrive/szakdolgozat/kepillesztes/preprocessed_2x.pkl"

with open(file_path, "wb") as file:
  result = run_pipeline(
      data=NanoReader(
          base_path=f"/content/datasets/{dataset_name}",
          flip_epic=[False, True]
      ),
      cellpose_model_path=f"/content/models/{model_name}",
      epic_params={
        "preprocessing_params": {
            "magnification": 2,
            "scaling": "MIC_10X",
            "range_lowerbound": 0.8,
            "drift_correction": {
                "threshold": 75,
                "filter_method": "mean"
            }
        },
        "localization_params": {
            "threshold_range": [25, 5000],
            "neighbourhood_size": 3
        }
      },
      only_process=True
  )
  pickle.dump(result, file)